In [1]:
import pandas as pd

# Import cleaned MetalliCan data

In [2]:
from utils.data_manipulations import build_activity_name, add_site_id

In [3]:
# Pre-processed production table
production_df = pd.read_excel(r'data/MetalliCan/sites_for_lci.xlsx', sheet_name='prod_data')

In [4]:
# Add activitiy_name to production_df and site_id
production_df['activity_name'] = production_df.apply(lambda row: build_activity_name(row, production_df), axis=1)
production_df = add_site_id(production_df)

In [5]:
# Normalized MetalliCan tables per ore processed
energy_ore_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/ore_normalization/energy_df.csv')
material_ore_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/ore_normalization/material_df.csv')
biosphere_ore_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/ore_normalization/biosphere_df.csv')

In [6]:
# Normalized MetalliCan tables per metal, economic allocation
energy_econ_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/economic_allocation/energy_df.csv')
material_econ_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/economic_allocation/material_df.csv')
biosphere_econ_df = pd.read_csv(r'data/MetalliCan/data_for_lci_initialization/economic_allocation/biosphere_df.csv')

In [7]:
# Removing rows with value_normalized is NaN in the biosphere dfs
biosphere_ore_df = biosphere_ore_df[~biosphere_ore_df['value_normalized'].isna()]
biosphere_econ_df['value_normalized'] = biosphere_econ_df['value_normalized'] / 1e6

In [8]:
# For all the dfs, multiply the value_normalized to 10e6 to have values per kg
energy_ore_df['value_normalized'] = energy_ore_df['value_normalized'] / 1e6
material_ore_df['value_normalized'] = material_ore_df['value_normalized'] / 1e6
biosphere_ore_df['value_normalized'] = biosphere_ore_df['value_normalized'] / 1e6
energy_econ_df['value_normalized'] = energy_econ_df['value_normalized'] / 1e6
material_econ_df['value_normalized'] = material_econ_df['value_normalized'] / 1e6
biosphere_econ_df['value_normalized'] = biosphere_econ_df['value_normalized'] / 1e6

# Keeping only relevant columns

In [9]:
technosphere_col = ['activity_name', 'functional_unit', 'site_id', 'subflow_type', 'value_normalized']
biosphere_col = ['activity_name', 'functional_unit', 'site_id', 'substance_name', 'value_normalized', 'unit']

In [10]:
energy_ore_df = energy_ore_df[technosphere_col]
material_ore_df = material_ore_df[technosphere_col]
biosphere_ore_df = biosphere_ore_df[biosphere_col]
energy_econ_df = energy_econ_df[technosphere_col]
material_econ_df = material_econ_df[technosphere_col]
biosphere_econ_df = biosphere_econ_df[biosphere_col]

In [11]:
# Add units to technosphere dfs
energy_ore_df['unit'] = 'MJ'
material_ore_df['unit'] = 't'
energy_econ_df['unit'] = 'MJ'
material_econ_df['unit'] = 't'

In [12]:
# Put energy_df and material_df together
technosphere_ore_df = pd.concat([energy_ore_df, material_ore_df], ignore_index=True)
technosphere_econ_df = pd.concat([energy_econ_df, material_econ_df], ignore_index=True)

In [13]:
# Add the province from the main_df to specify electricity location later
technosphere_ore_df = technosphere_ore_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')
technosphere_econ_df = technosphere_econ_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')
biosphere_ore_df = biosphere_ore_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')
biosphere_econ_df = biosphere_econ_df.merge(production_df[['site_id', 'province']], on=['site_id'], how='left')

In [14]:
technosphere_ore_df.to_csv(r'technosphere_ore_df.csv', index=False)

# Map MetalliCan flows to EI and RI flows

In [15]:
from utils.conversion_functions import map_technosphere_to_ecoinvent, map_biosphere_to_ecoinvent
from utils.constants import CA_provinces

## Technosphere flows

In [16]:
mapping_technosphere = pd.read_excel(r'data/Mappings/MAPPINGS_RI.xlsx', sheet_name='technosphere')

In [17]:
technosphere_ore_df

,activity_name,functional_unit,site_id,subflow_type,value_normalized,unit,province
0,"Au, Underground mining and beneficiation at Br...",Ore processed,BC-MAIN-857b7b89,Acetylene,1.112991e-07,MJ,British Columbia
1,"Au and Cu, Open-pit mining and beneficiation a...",Ore processed,BC-MAIN-8eb8be0d,Acetylene,1.755926e-09,MJ,British Columbia
2,"Au and Ag, Underground mining and beneficiatio...",Ore processed,GRP-147b3123,Ammonium nitrate,1.315121e-08,MJ,Ontario
3,"Au and Ag, Underground mining and beneficiatio...",Ore processed,QC-MAIN-02884fb5,ANFO,1.480561e-06,MJ,Quebec
4,"Au, Underground mining and beneficiation at Br...",Ore processed,BC-MAIN-857b7b89,Aviation fuel,4.378079e-04,MJ,British Columbia
...,...,...,...,...,...,...,...
227,"Au, Underground mining and beneficiation at Is...",Ore processed,ON-MAIN-687b8c8d,Total blasting agents used e.g. ANFO,1.107725e-09,t,Ontario
228,"Au, Underground mining and beneficiation at Yo...",Ore processed,ON-MAIN-7607a50e,Total blasting agents used e.g. ANFO,9.032167e-10,t,Ontario
229,"Au, Underground mining and beneficiation at Is...",Ore processed,ON-MAIN-687b8c8d,Total sodium cyanide used,5.580764e-10,t,Ontario
230,"Au, Underground mining and beneficiation at Yo...",Ore processed,ON-MAIN-7607a50e,Total sodium cyanide used,6.795233e-10,t,Ontario


In [18]:
# Apply the function
mapped_technosphere_ore_df = map_technosphere_to_ecoinvent(technosphere_ore_df, mapping_technosphere, CA_provinces)
mapped_technosphere_econ_df = map_technosphere_to_ecoinvent(technosphere_econ_df, mapping_technosphere, CA_provinces)

⚠️ Les flux suivants n'ont pas trouvé de correspondance dans Ecoinvent:
 - Energy use
 - Other
 - Surface/underground emulsion & ANFO
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Ammonium nitrate)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Dynamite)
⚠️ Les flux suivants n'ont pas trouvé de correspondance dans Ecoinvent:
 - Energy use
 - Other
 - Surface/underground emulsion & ANFO
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Ammonium nitrate)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Ammonium nitrate)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Dynamite)
⚠️ Pas de conversion définie pour MJ → kilogram (flux: Dynamite)


In [19]:
# Drop rows where ecoinvent_flow_name is "No mapping" and Amount is NaN for now
mapped_technosphere_ore_df = mapped_technosphere_ore_df[(mapped_technosphere_ore_df["Activity"] != "No mapping") & (~mapped_technosphere_ore_df["Amount"].isna())]
mapped_technosphere_econ_df = mapped_technosphere_econ_df[(mapped_technosphere_econ_df["Activity"] != "No mapping") & (~mapped_technosphere_econ_df["Amount"].isna())]

In [20]:
mapped_technosphere_econ_df

,activity_name,functional_unit,site_id,subflow_type,Activity,Product,value_normalized,unit,Amount,Unit,Location,province,Database
0,"Au, Underground mining and beneficiation at Br...","Au, usable ore",BC-MAIN-857b7b89,Acetylene,consumption market for acetylene,acetylene,2.048295e-02,MJ,4.267281e-04,kilogram,CA,British Columbia,Regioinvent
1,"Au and Cu, Open-pit mining and beneficiation a...","Au, usable ore",BC-MAIN-8eb8be0d,Acetylene,consumption market for acetylene,acetylene,3.130155e-03,MJ,6.521156e-05,kilogram,CA,British Columbia,Regioinvent
2,"Au and Cu, Open-pit mining and beneficiation a...","Cu, usable ore",BC-MAIN-8eb8be0d,Acetylene,consumption market for acetylene,acetylene,4.170064e-07,MJ,8.687633e-09,kilogram,CA,British Columbia,Regioinvent
3,"Au and Ag, Underground mining and beneficiatio...","Au, usable ore",GRP-147b3123,Ammonium nitrate,consumption market for ammonium nitrate,ammonium nitrate,4.999863e-03,MJ,4.999863e-03,kilogram,CA,Ontario,Regioinvent
4,"Au and Ag, Underground mining and beneficiatio...","Ag, usable ore",GRP-147b3123,Ammonium nitrate,consumption market for ammonium nitrate,ammonium nitrate,6.488424e-05,MJ,6.488424e-05,kilogram,CA,Ontario,Regioinvent
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,"Au, Underground mining and beneficiation at Yo...","Au, usable ore",ON-MAIN-7607a50e,Total blasting agents used e.g. ANFO,"consumption market for explosive, tovex","explosive, tovex",4.515170e-04,t,4.515170e-01,kilogram,CA,Ontario,Regioinvent
439,"Au, Underground mining and beneficiation at Is...","Au, usable ore",ON-MAIN-687b8c8d,Total sodium cyanide used,consumption market for sodium cyanide,sodium cyanide,5.994617e-05,t,5.994617e-02,kilogram,CA,Ontario,Regioinvent
440,"Au, Underground mining and beneficiation at Yo...","Au, usable ore",ON-MAIN-7607a50e,Total sodium cyanide used,consumption market for sodium cyanide,sodium cyanide,3.396930e-04,t,3.396930e-01,kilogram,CA,Ontario,Regioinvent
441,"Au and Ag, Underground mining and beneficiatio...","Au, usable ore",QC-MAIN-02884fb5,Transmission oil,consumption market for lubricating oil,lubricating oil,2.785936e-06,t,2.785936e-03,kilogram,CA,Quebec,Regioinvent


In [21]:
technosphere_col_for_lci = ['site_id', 'activity_name', 'functional_unit', 'Amount', 'Activity', 'Product', 'Unit', 'Location', 'Database']

In [22]:
mapped_technosphere_ore_df = mapped_technosphere_ore_df[technosphere_col_for_lci]
mapped_technosphere_econ_df = mapped_technosphere_econ_df[technosphere_col_for_lci]

## Biosphere flows mapping

In [23]:
mapping_biosphere = pd.read_excel(r'data/Mappings/MAPPINGS_RI.xlsx', sheet_name='biosphere')

In [24]:
mapped_biosphere_ore_df = map_biosphere_to_ecoinvent(biosphere_ore_df, mapping_biosphere, CA_provinces)
mapped_biosphere_econ_df = map_biosphere_to_ecoinvent(biosphere_econ_df, mapping_biosphere, CA_provinces)

Index(['activity_name', 'functional_unit', 'site_id', 'substance_name',
       'value_normalized', 'unit', 'province', 'Type', 'substance_id',
       'compartment_name', 'release_pathway', 'flow_direction',
       'MetalliCan_unit', 'DB_to_map', 'Flow name', 'Compartments', 'Unit',
       'Comment', 'Alternatives'],
      dtype='object')
⚠️ 6 biosphere flows could not be mapped to Ecoinvent:
   - 1-Nitropyrene
   - Quinoline
   - Ethylene glycol
   - nan
   - N2O
   - PFCs
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for kg → nan (flow: 1-Nitropyrene)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠️ No conversion defined for kg → nan (flow: Quinoline)
⚠️ No conversion defined for g teq → kilogram (flow: Dioxins and furans - total)
⚠

In [25]:
# Drop rows where ecoinvent_flow_name is "No mapping" and Amount is NaN for now
mapped_biosphere_ore_df = mapped_biosphere_ore_df[(mapped_biosphere_ore_df["Flow Name"] != "No mapping") & (~mapped_biosphere_ore_df["Amount"].isna())]
mapped_biosphere_econ_df = mapped_biosphere_econ_df[(mapped_biosphere_econ_df["Flow Name"] != "No mapping") & (~mapped_biosphere_econ_df["Amount"].isna())]

In [26]:
biosphere_col_for_lci = ['site_id', 'activity_name', 'functional_unit', 'Amount', 'Unit', 'Flow Name', 'Compartments', 'Database']

In [27]:
mapped_biosphere_ore_df = mapped_biosphere_ore_df[biosphere_col_for_lci]
mapped_biosphere_econ_df = mapped_biosphere_econ_df[biosphere_col_for_lci]

In [28]:
production_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/production_df.csv', index=False)

In [29]:
mapped_biosphere_ore_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/biosphere_ore_df.csv', index=False)
mapped_biosphere_econ_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/biosphere_econ_df.csv', index=False)

In [30]:
mapped_technosphere_ore_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/technosphere_ore_df.csv', index=False)
mapped_technosphere_econ_df.to_csv(r'data/MetalliCan/data_for_lci_initialization/technosphere_econ_df.csv', index=False)

In [31]:
# Add province and commodities from NRCan to production table
mapped_biosphere_ore_df = mapped_biosphere_ore_df.merge(production_df[['site_id', 'province', 'commodities']], on=['site_id'], how='left')
mapped_biosphere_econ_df = mapped_biosphere_econ_df.merge(production_df[['site_id', 'province', 'commodities']], on=['site_id'], how='left')

In [32]:
mapped_technosphere_econ_df

,site_id,activity_name,functional_unit,Amount,Activity,Product,Unit,Location,Database
0,BC-MAIN-857b7b89,"Au, Underground mining and beneficiation at Br...","Au, usable ore",4.267281e-04,consumption market for acetylene,acetylene,kilogram,CA,Regioinvent
1,BC-MAIN-8eb8be0d,"Au and Cu, Open-pit mining and beneficiation a...","Au, usable ore",6.521156e-05,consumption market for acetylene,acetylene,kilogram,CA,Regioinvent
2,BC-MAIN-8eb8be0d,"Au and Cu, Open-pit mining and beneficiation a...","Cu, usable ore",8.687633e-09,consumption market for acetylene,acetylene,kilogram,CA,Regioinvent
3,GRP-147b3123,"Au and Ag, Underground mining and beneficiatio...","Au, usable ore",4.999863e-03,consumption market for ammonium nitrate,ammonium nitrate,kilogram,CA,Regioinvent
4,GRP-147b3123,"Au and Ag, Underground mining and beneficiatio...","Ag, usable ore",6.488424e-05,consumption market for ammonium nitrate,ammonium nitrate,kilogram,CA,Regioinvent
...,...,...,...,...,...,...,...,...,...
438,ON-MAIN-7607a50e,"Au, Underground mining and beneficiation at Yo...","Au, usable ore",4.515170e-01,"consumption market for explosive, tovex","explosive, tovex",kilogram,CA,Regioinvent
439,ON-MAIN-687b8c8d,"Au, Underground mining and beneficiation at Is...","Au, usable ore",5.994617e-02,consumption market for sodium cyanide,sodium cyanide,kilogram,CA,Regioinvent
440,ON-MAIN-7607a50e,"Au, Underground mining and beneficiation at Yo...","Au, usable ore",3.396930e-01,consumption market for sodium cyanide,sodium cyanide,kilogram,CA,Regioinvent
441,QC-MAIN-02884fb5,"Au and Ag, Underground mining and beneficiatio...","Au, usable ore",2.785936e-03,consumption market for lubricating oil,lubricating oil,kilogram,CA,Regioinvent


# LCI creation

In [33]:
from core.lci_database_builder import LCIDatabaseBuilder

## Ore processed basism

In [34]:
# Step 1 — initialize the builder
builder_op = LCIDatabaseBuilder(db_name='metallican_lci', project_name='metallican')

📂 Active Brightway project: metallican
🆕 Database 'metallican_lci' created.


In [35]:
# Step 2 — create the activity shells from the main dataframe
builder_op.build_lci_entries(df=mapped_biosphere_ore_df)
print(len(builder_op.lcis))

✅ Created 33 base LCI activities with production exchanges.
33


In [36]:
# Step 3a — Populate with the technosphere exchanges
builder_op.populate_technosphere_exchanges(technosphere_df=mapped_technosphere_ore_df)

⚙️ Populating technosphere exchanges
   ✅ Cached 218246 activities from Regioinvent
   ✅ Cached 20769 activities from ecoinvent-3.10-cutoff regionalized
✅ Added 246 technosphere exchanges.


In [37]:
# Step 3b — Populate with the biosphere exchanges
builder_op.populate_biosphere_exchanges(biosphere_df=mapped_biosphere_ore_df)

🌱 Populating biosphere exchanges
   ✅ Cached 110559 biosphere flows from biosphere3_spatialized_flows
   ✅ Cached 4362 biosphere flows from biosphere3
✅ Added 7889 biosphere exchanges.


In [38]:
# Step 4 - Consolidate duplicate flows
builder_op.consolidate_exchanges()

🧮 Consolidation: 8168 → 849 exchanges (summed duplicates).


In [39]:
builder_op.write_to_database()

🧱 Writing 33 activities to database 'metallican_lci'...
✅ Database 'metallican_lci' processed successfully with 33 activities.


## Usable ore basis

In [40]:
# Step 1 — initialize the builder
builder_uo = LCIDatabaseBuilder(db_name='metallican_lci', project_name='metallican')

📂 Active Brightway project: metallican
✅ Using existing database 'metallican_lci'.


In [41]:
# Step 2 — create the activity shells from the main dataframe
builder_uo.build_lci_entries(df=mapped_biosphere_econ_df)
print(len(builder_uo.lcis))

✅ Created 59 base LCI activities with production exchanges.
59


In [42]:
# Step 3a — Populate with the technosphere exchanges
builder_uo.populate_technosphere_exchanges(technosphere_df=mapped_technosphere_econ_df)

⚙️ Populating technosphere exchanges
   ✅ Cached 218246 activities from Regioinvent
   ✅ Cached 20769 activities from ecoinvent-3.10-cutoff regionalized
✅ Added 424 technosphere exchanges.


In [43]:
# Step 3b — Populate with the biosphere exchanges
builder_uo.populate_biosphere_exchanges(biosphere_df=mapped_biosphere_econ_df)

🌱 Populating biosphere exchanges
   ✅ Cached 110559 biosphere flows from biosphere3_spatialized_flows
   ✅ Cached 4362 biosphere flows from biosphere3
✅ Added 13111 biosphere exchanges.


In [44]:
# Step 4 - Consolidate duplicate flows
builder_uo.consolidate_exchanges()

🧮 Consolidation: 13594 → 1451 exchanges (summed duplicates).


In [45]:
builder_uo.write_to_database()

🧱 Writing 59 activities to database 'metallican_lci'...
✅ Database 'metallican_lci' processed successfully with 92 activities.
